In [ ]:
import requests
import json
from api_key import api_key
import pandas as pd
import time

In [ ]:
class Stock():
    def __init__(self, symbol,api_key):
        self.symbol=symbol
        self.api_key=api_key
        self.api_output=self.get_api_data()
        self.weight=1
        
    def get_api_data(self):
        """make api call"""
        #Would want to handle non-200 responses 
        r=requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+self.symbol+'&outputsize=full&apikey='+self.api_key)
        return r
    
    def get_stock_df(self):
        """Create a dataframe indexed by day wwhich has share price info"""
        stock_hist=json.loads(self.api_output.content)
        stock_df=pd.DataFrame(stock_hist['Time Series (Daily)']).transpose()
        stock_df=stock_df.astype(float)
        stock_df['9. symbol']=stock_hist['Meta Data']['2. Symbol']+' '
        return stock_df
    

class Index():
    #Need to do the API calls in the index because it's limited to 5/minute and 500/day
    def __init__(self):
        self.sp500_df=self.get_sp_df()
        self.sp500_prices={}
        self.api_all_stocks()
    def get_sp_df(self):
        #Need a better source for SP500 financials like mkcp
        sp500=requests.get('https://datahub.io/core/s-and-p-500-companies-financials/r/constituents-financials.json')
        sp500_df=pd.DataFrame(json.loads(sp500.content))
        sp500_df['Shares']=sp500_df['Market Cap']/sp500_df['Price']
        return sp500_df
    def sample_sp(self,num_sample):
        return self.sp500_df.sample(num_sample)
    
    def api_all_stocks(self):
        symb_list=self.sp500_df['Symbol'].tolist()
        for i in symb_list:
            self.sp500_prices[i]=Stock(i, api_key)
            time.sleep(15)

    
    
class Portfolio():
    def __init__(self, symbol_list):
        self.symbol_list=symbol_list
        self.Stocks=self.get_all_stocks()
    def get_all_stocks(self):
        Stocks=[]
        for symbol in self.symbol_list:
            Stocks.append(Stock(symbol,api_key))
        return Stocks
    def total_portfolio_df(self):
        loop_check=0
        for stock in self.Stocks:
            if loop_check==0:
                agg_df=stock.weight*stock.get_stock_df()
            else:
                agg_df+=stock.get_stock_df()
        

In [ ]:
sp500=Index()

In [ ]:

#sample_stocks=sp500.sample_sp(10)
#sample_stocks

import pickle
pickle.dump( sp500, open( "save.p", "wb" ) )

In [ ]:
sp500.sp500_prices['DIS'].api_output.content

In [ ]:
port=Portfolio(sample_stocks['Symbol'].tolist())

In [ ]:
print(port.Stocks[5].symbol,port.Stocks[5].api_output.content)

In [ ]:
port.total_portfolio_df()

In [ ]:
stock1=Stock('DIS',api_key)
stock2=Stock('T',api_key)
combined=stock1.get_stock_df().join(stock2.get_stock_df(),lsuffix=' '+stock1.symbol,rsuffix=' '+stock2.symbol)

In [ ]:
stock1.get_stock_df()+stock2.get_stock_df()

In [ ]:
#combined

In [ ]:
combined[['5. adjusted close '+stock1.symbol,'5. adjusted close '+stock2.symbol]].astype(float).corr()

In [ ]:


stock_list=sp500_df.sample(5)['Symbol'].tolist()
index=Portfolio(stock_list)
index.Stocks[0].get_stock_df()

In [ ]:
z=Index()

In [ ]:
z.

In [ ]:
z=Stock('DIS',api_key)
z.get_stock_df()